In [0]:
import numpy as np
import pandas as pd
import os

In [0]:
train_file = pd.read_csv("train.csv")
dances = train_file.target.unique()
train_file.columns

Index(['Image', 'target'], dtype='object')

In [0]:
current_directory = os.getcwd()
for i in dances:
    final_directory = os.path.join(current_directory, 'final_data', i)
    if not os.path.exists(final_directory):
       os.makedirs(final_directory)

In [0]:
images = train_file['Image']
target = train_file['target']

In [0]:
import shutil

In [0]:
for i in range(0, len(images)):
    shutil.move(current_directory + '/dataset/train/'+str(images[i]), 
                                                          current_directory+'/final_data/'+'/'+target[i]+'/'+str(images[i]))
    print(images[i], target[i])

FileNotFoundError: [Errno 2] No such file or directory: '/home/satyam/Desktop/hackerearth_challenge/dance/deep_learning/dataset/train/96.jpg'

In [0]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [0]:
IMAGE_SIZE = [224, 224]

In [104]:
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

94658560/94653016 [==============================] - 1s 0us/step


In [0]:
# don't train existing weights
for layer in resnet.layers:
  layer.trainable = False

In [0]:
# useful for getting number of classes
folders = glob('final_data/*')

In [0]:
# our layers - you can add more if you want
x = Flatten()(resnet.output)
x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

In [0]:
model = Model(inputs=resnet.input, outputs=prediction)

In [109]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [0]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [111]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('final_data/',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

#test_set = test_datagen.flow_from_directory('/',
#                                             target_size = (224, 224),
#                                             batch_size = 32)

Found 364 images belonging to 8 classes.


In [113]:
r = model.fit_generator(
  training_set,
  #validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set)
 # validation_steps=len(test_set)
)

Epoch 1/20
12/12 [==============================] - 6s 469ms/step - loss: 22.7633 - accuracy: 0.4670
Epoch 2/20
12/12 [==============================] - 6s 461ms/step - loss: 4.3691 - accuracy: 0.7363
Epoch 3/20
12/12 [==============================] - 6s 461ms/step - loss: 1.4756 - accuracy: 0.8736
Epoch 4/20
12/12 [==============================] - 6s 459ms/step - loss: 0.9316 - accuracy: 0.9203
Epoch 5/20
12/12 [==============================] - 5s 457ms/step - loss: 0.7062 - accuracy: 0.9423
Epoch 6/20
12/12 [==============================] - 5s 455ms/step - loss: 0.4087 - accuracy: 0.9423
Epoch 7/20
12/12 [==============================] - 5s 454ms/step - loss: 0.2158 - accuracy: 0.9670
Epoch 8/20
12/12 [==============================] - 5s 456ms/step - loss: 0.2010 - accuracy: 0.9698
Epoch 9/20
12/12 [==============================] - 6s 465ms/step - loss: 0.0996 - accuracy: 0.9808
Epoch 10/20
12/12 [==============================] - 6s 464ms/step - loss: 0.1184 - accuracy: 0.980

In [0]:
import tensorflow as tf
from keras.models import load_model

model.save('my_model.h5')

In [0]:
from keras.preprocessing import image
import operator
def imgpred(imgname):
  test_image= image.load_img('test/'+str(imgname), target_size = (224, 224)) 
  test_image = image.img_to_array(test_image)
  test_image = np.expand_dims(test_image, axis = 0)
  #test_image = test_image.reshape(224, 224)
  result = model.predict(test_image) 
  r = result.reshape(-1,1)
  index, value = max(enumerate(r), key=operator.itemgetter(1))
  d = ['bharatanatyam', 'kathak', 'kathakali','kuchipudi', 'manipuri', 'mohiniyattam',
      'odissi', 'sattriya']
  return d[index]

In [0]:
preds = []
timages = []
for i in (testimages):
  answer = imgpred(i)
  timages.append(i)
  preds.append(answer)

In [0]:
rows = zip(timages, preds)
import csv
with open('finalpred.csv', "w") as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)

In [0]:
import pandas as pd
df = pd.read_csv('finalpred.csv', header=None)
df.rename(columns={0: 'Image', 1: 'target'}, inplace=True)
df.to_csv('finalpred.csv', index=False)